# Visualizing harmful PM<sub>2.5</sub> levels in the US by county

In [13]:
# %pip install pandas geopandas folium matplotlib mapclassify

In [ ]:
import pandas as pd
import geopandas as gpd

In [3]:
# download the data from US NIH (https://hdpulse.nimhd.nih.gov/data-portal/physical/table?age=001&age_options=ageall_1&demo=234&demo_options=air_pollution_1&physicaltopic=002&physicaltopic_options=physical_2&race=00&race_options=raceall_1&sex=0&sex_options=sexboth_1&statefips=99&statefips_options=area_states)

county_pm25: pd.DataFrame = pd.read_csv(
    "HDPulse_data_export.csv",
    skiprows=5,
)

In [4]:
county_pm25

,County,FIPS,Micrograms per cubic meter (PM2.5)(2)
0,United States,0.0,7.4
1,"San Bernardino County, California",6071.0,15.6
2,"Fairbanks North Star, Alaska",2090.0,15.5
3,"Allegheny County, Pennsylvania",42003.0,14.1
4,"San Diego County, California",6073.0,13.8
...,...,...,...
3146,Notes:,NaN,NaN
3147,Source: National Environmental Public Health T...,NaN,NaN
3148,Average daily density of fine particulate matt...,NaN,NaN
3149,Some data are not available or suppressed due ...,NaN,NaN


In [5]:
county_pm25_processed: pd.DataFrame = (
    county_pm25.assign(
        # make PM2.5 reading a float
        pm25_ug_per_m3=lambda x: pd.to_numeric(arg=x[x.keys()[-1]], errors="coerce"),
        # convert FIPS to a 5-digit string
        FIPS=lambda x: pd.to_numeric(x["FIPS"]),
    )
    .dropna(
        # drop rows with missing PM2.5 readings
        subset=[
            "FIPS",
            "pm25_ug_per_m3",
        ],
    )
    .assign(
        FIPS=lambda x: x["FIPS"].astype(int).astype(str).str.zfill(5),
    )
)

In [6]:
# optional sense check
county_pm25_processed

,County,FIPS,Micrograms per cubic meter (PM2.5)(2),pm25_ug_per_m3
0,United States,00000,7.4,7.4
1,"San Bernardino County, California",06071,15.6,15.6
2,"Fairbanks North Star, Alaska",02090,15.5,15.5
3,"Allegheny County, Pennsylvania",42003,14.1,14.1
4,"San Diego County, California",06073,13.8,13.8
...,...,...,...,...
3111,"Custer County, South Dakota",46033,2.6,2.6
3112,"Apache County, Arizona",04001,2.5,2.5
3113,"Campbell County, Wyoming",56005,2.4,2.4
3114,"Converse County, Wyoming",56009,2.2,2.2


In [7]:
# download us county shape files from https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
counties: gpd.GeoDataFrame = gpd.read_file(
    "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip"
)

In [8]:
counties_processed: gpd.GeoDataFrame = counties.assign(
    FIPS=lambda x: x["STATEFP"] + x["COUNTYFP"],
)

In [9]:
# optional sense check
counties_processed

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,FIPS
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069
...,...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079


In [10]:
# merge the two dataframes
counties_w_pm25 = counties_processed.merge(
    right=county_pm25_processed,
    on="FIPS",
    how="left",
)

In [11]:
# optional sense check
counties_w_pm25

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,FIPS,County,Micrograms per cubic meter (PM2.5)(2),pm25_ug_per_m3
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053...",21007,"Ballard County, Kentucky",8.7,8.7
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,"Bourbon County, Kentucky",8.5,8.5
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074...",21031,"Butler County, Kentucky",8.3,8.3
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646...",21065,"Estill County, Kentucky",7.9,7.9
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450...",21069,"Fleming County, Kentucky",8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4...",31073,"Gosper County, Nebraska",5.5,5.5
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667...",39075,"Holmes County, Ohio",9.2,9.2
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499...",48171,"Gillespie County, Texas",8.2,8.2
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872...",55079,"Milwaukee County, Wisconsin",8.7,8.7


In [ ]:
# display the data on map by level of PM2.5
counties_w_pm25.explore(
    column="pm25_ug_per_m3",
)